In [7]:
#imports

import pandas as pd
import numpy as np
import os

import datetime

from prepare import zillow_pipeline

In [18]:
df = zillow_pipeline()

In [4]:
df.head()

,bedrooms,bathrooms,area,value,year,county,state
0,4,3.5,3100,1023282,1998,Orange,CA
1,2,1.0,1465,464000,1967,Ventura,CA
2,3,2.0,1243,564778,1962,Orange,CA
3,4,3.0,2376,145143,1970,Los Angeles,CA
4,4,3.0,2962,773303,1950,Los Angeles,CA


In [5]:
df.shape

(52246, 7)

In [19]:
# Assuming df is your original DataFrame
df['total_rooms'] = df['bedrooms'] + df['bathrooms']

current_year = datetime.datetime.now().year
df['age'] = current_year - df['year']

#df['price_per_sqft'] = df['tax_value'] / df['area']

#df['tax_rate'] = df['taxamount'] / df['tax_value']

#df = pd.get_dummies(df, columns=['fips'], prefix='county')

df['bdrm_area_ratio'] = df['area'] / df['bedrooms']

df['bath_bdrm_ratio'] = df['bathrooms'] / df['bedrooms']

In [20]:
df.head()

,bedrooms,bathrooms,area,value,year,county,state,total_rooms,age,sqft_per_bdrm,bath_bdrm_ratio
0,4,3.5,3100,1023282,1998,Orange,CA,7.5,25,775.000000,0.875000
1,2,1.0,1465,464000,1967,Ventura,CA,3.0,56,732.500000,0.500000
2,3,2.0,1243,564778,1962,Orange,CA,5.0,61,414.333333,0.666667
3,4,3.0,2376,145143,1970,Los Angeles,CA,7.0,53,594.000000,0.750000
4,4,3.0,2962,773303,1950,Los Angeles,CA,7.0,73,740.500000,0.750000


In [21]:
# Define the age group categories
age_bins = [-1, 4, 9, 19, 39, 59, 79, 200]
age_labels = ['Newest', 'Very_New', 'New', 'Mid', 'Old', 'Very_Old', 'Oldest']

# Create the "age_group" column
df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels)

df.head()

,bedrooms,bathrooms,area,value,year,county,state,total_rooms,age,sqft_per_bdrm,bath_bdrm_ratio,age_group
0,4,3.5,3100,1023282,1998,Orange,CA,7.5,25,775.000000,0.875000,Mid
1,2,1.0,1465,464000,1967,Ventura,CA,3.0,56,732.500000,0.500000,Old
2,3,2.0,1243,564778,1962,Orange,CA,5.0,61,414.333333,0.666667,Very_Old
3,4,3.0,2376,145143,1970,Los Angeles,CA,7.0,53,594.000000,0.750000,Old
4,4,3.0,2962,773303,1950,Los Angeles,CA,7.0,73,740.500000,0.750000,Very_Old


In [24]:
# Find percentiles for square footage
percentiles = df['area'].quantile([0.01, 0.05, 0.20, 0.80, 0.95, 0.99])

# Define the categories
df['size_group'] = pd.cut(
    df['area'],
    bins=[0, percentiles[0.01], percentiles[0.05], percentiles[0.20], percentiles[0.80], percentiles[0.95], percentiles[0.99], df['area'].max()],
    labels=['Tiny', 'Small', 'Medium', 'Average', 'Large', 'Very Large', 'Largest']
)

In [25]:
df.head()

,bedrooms,bathrooms,area,value,year,county,state,total_rooms,age,sqft_per_bdrm,bath_bdrm_ratio,age_group,size_group
0,4,3.5,3100,1023282,1998,Orange,CA,7.5,25,775.000000,0.875000,Mid,Large
1,2,1.0,1465,464000,1967,Ventura,CA,3.0,56,732.500000,0.500000,Old,Average
2,3,2.0,1243,564778,1962,Orange,CA,5.0,61,414.333333,0.666667,Very_Old,Average
3,4,3.0,2376,145143,1970,Los Angeles,CA,7.0,53,594.000000,0.750000,Old,Average
4,4,3.0,2962,773303,1950,Los Angeles,CA,7.0,73,740.500000,0.750000,Very_Old,Large


In [26]:
# Count the occurrences of each category in the 'size_group' column
size_group_counts = df['size_group'].value_counts().sort_index()

# Print the category names and their corresponding counts
for category, count in size_group_counts.items():
    print(f'{category}: {count}')

Tiny: 523
Small: 2114
Medium: 7829
Average: 31351
Large: 7819
Very Large: 2088
Largest: 522


In [27]:

# Group the DataFrame by 'size_group' and calculate the minimum and maximum values for 'area'
size_group_ranges = df.groupby('size_group')['area'].agg(['min', 'max'])

# Print the size_group ranges
for size_group, (min_area, max_area) in size_group_ranges.iterrows():
    print(f'{size_group}: {min_area}-{max_area} sqft')

Tiny: 152-699 sqft
Small: 700-896 sqft
Medium: 897-1200 sqft
Average: 1201-2510 sqft
Large: 2511-3730 sqft
Very Large: 3731-5497 sqft
Largest: 5500-21929 sqft


In [29]:
# Calculate the percentiles
percentiles = df['area'].quantile([0.01, 0.05, 0.32, 0.68, 0.95, 0.99])

# Define the categories
df['sqft_category'] = pd.cut(
    df['area'],
    bins=[0, percentiles[0.01], percentiles[0.05], percentiles[0.32], percentiles[0.68], percentiles[0.95], percentiles[0.99], df['area'].max()],
    labels=['smallest', 'very_small', 'small', 'Average', 'large', 'very_large', 'Largest']
)


In [30]:
df.head()

,bedrooms,bathrooms,area,value,year,county,state,total_rooms,age,sqft_per_bdrm,bath_bdrm_ratio,age_group,size_group,sqft_category
0,4,3.5,3100,1023282,1998,Orange,CA,7.5,25,775.000000,0.875000,Mid,Large,Large
1,2,1.0,1465,464000,1967,Ventura,CA,3.0,56,732.500000,0.500000,Old,Average,Average
2,3,2.0,1243,564778,1962,Orange,CA,5.0,61,414.333333,0.666667,Very_Old,Average,Medium
3,4,3.0,2376,145143,1970,Los Angeles,CA,7.0,53,594.000000,0.750000,Old,Average,Large
4,4,3.0,2962,773303,1950,Los Angeles,CA,7.0,73,740.500000,0.750000,Very_Old,Large,Large


In [31]:
# Count the occurrences of each category in the 'size_group' column
size_group_counts = df['sqft_category'].value_counts().sort_index()

# Print the category names and their corresponding counts
for category, count in size_group_counts.items():
    print(f'{category}: {count}')

Tiny: 523
Small: 2114
Medium: 14097
Average: 18797
Large: 14105
very_large: 2088
Largest: 522


In [32]:

# Group the DataFrame by 'size_group' and calculate the minimum and maximum values for 'area'
size_group_ranges = df.groupby('sqft_category')['area'].agg(['min', 'max'])

# Print the size_group ranges
for size_group, (min_area, max_area) in size_group_ranges.iterrows():
    print(f'{size_group}: {min_area}-{max_area} sqft')

Tiny: 152-699 sqft
Small: 700-896 sqft
Medium: 897-1374 sqft
Average: 1375-2072 sqft
Large: 2073-3730 sqft
very_large: 3731-5497 sqft
Largest: 5500-21929 sqft


In [ ]:
Tiny: 152-699 sqft
Small: 700-896 sqft
Medium: 897-1200 sqft
Average: 1201-2510 sqft
Large: 2511-3730 sqft
Very Large: 3731-5497 sqft
Largest: 5500-21929 sqft